In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cufflinks as cf
import re
import seaborn as sns

import cufflinks as cf
from sklearn import set_config
from sklearn import svm

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression

#cf.go_offline()
pd.set_option('display.max_columns', 500)
# Environment setup
cf.go_offline()
set_config(display="diagram")



In [10]:
s1 = pd.read_excel('/home/daniel/Documentos/NFL/NFL_2011.xlsx')
s2 = pd.read_excel('/home/daniel/Documentos/NFL/NFL_2012.xlsx')
s3 = pd.read_excel('/home/daniel/Documentos/NFL/NFL_2013.xlsx')
s4 = pd.read_excel('/home/daniel/Documentos/NFL/NFL_2014.xlsx')
s5 = pd.read_excel('/home/daniel/Documentos/NFL/NFL_2015.xlsx')
s6 = pd.read_excel('/home/daniel/Documentos/NFL/NFL_2016.xlsx')
s7 = pd.read_excel('/home/daniel/Documentos/NFL/NFL_2017.xlsx')
s8 = pd.read_excel('/home/daniel/Documentos/NFL/NFL_2018.xlsx')
s9 = pd.read_excel('/home/daniel/Documentos/NFL/NFL_2019.xlsx')
s10 = pd.read_excel('/home/daniel/Documentos/NFL/NFL_2020.xlsx')
s11 = pd.read_excel('/home/daniel/Documentos/NFL/NFL_2021.xlsx')

In [11]:
s1

,Rk,Player,Tm,Age,G,GS,Cmp,Att,Cmp%,Yds,TD,Int,1D,Y/G,Sk
0,1,Drew Brees,NOR,32,16,16,468,657,71.2,5476,46,14,279,342.3,24
1,2,Tom Brady,NWE,34,16,16,401,611,65.6,5235,39,12,262,327.2,32
2,3,Matthew Stafford,DET,23,16,16,421,663,63.5,5038,41,16,243,314.9,36
3,4,Eli Manning,NYG,30,16,16,359,589,61.0,4933,29,16,218,308.3,28
4,5,Aaron Rodgers,GNB,28,15,15,343,502,68.3,4643,45,6,209,309.5,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,92,Adam Podlesh,CHI,28,16,0,0,1,0.0,0,0,0,0,0.0,0
92,93,Bernard Scott,CIN,27,16,1,0,1,0.0,0,0,0,0,0.0,0
93,94,Brad Smith,BUF,28,15,5,0,1,0.0,0,0,1,0,0.0,1
94,95,Micheal Spurlock,TAM,28,12,0,0,1,0.0,0,0,0,0,0.0,0


In [12]:
s1_disc = ['Player','Tm']
s1_cont = s1.columns.drop(s1_disc)

In [13]:
seasons = [s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11]

In [14]:
seasons[0].isna().mean()

Rk        0.0
Player    0.0
Tm        0.0
Age       0.0
G         0.0
GS        0.0
Cmp       0.0
Att       0.0
Cmp%      0.0
Yds       0.0
TD        0.0
Int       0.0
1D        0.0
Y/G       0.0
Sk        0.0
dtype: float64

In [15]:
for variable in s1_disc:
    display(s1[variable].iplot(kind="hist", theme="solar", title=f"Histogram for {variable}"))

None

None

In [56]:
def TD_Max(seasons,año):
    column = seasons["TD"]
    max_value = column.max()
    print("La mayor cantidad de TD en la temporada",año,"fue:",max_value)
    
for i in range(0,11):
    año = 2010
    añoN = año+i
    TD_Max(seasons[i],añoN)

La mayor cantidad de TD en la temporada 2010 fue: 46
La mayor cantidad de TD en la temporada 2011 fue: 43
La mayor cantidad de TD en la temporada 2012 fue: 55
La mayor cantidad de TD en la temporada 2013 fue: 40
La mayor cantidad de TD en la temporada 2014 fue: 36
La mayor cantidad de TD en la temporada 2015 fue: 40
La mayor cantidad de TD en la temporada 2016 fue: 34
La mayor cantidad de TD en la temporada 2017 fue: 50
La mayor cantidad de TD en la temporada 2018 fue: 36
La mayor cantidad de TD en la temporada 2019 fue: 48
La mayor cantidad de TD en la temporada 2020 fue: 43


In [ ]:
def TD_Max(seasons,año):
    column = seasons["TD"]
    max_value = column.max()
    print("El QB con más TD en el año",año,"fue:",max_value)
    
for i in range(0,11):
    año = 2010
    añoN = año+i
    TD_Max(seasons[i],añoN)

In [57]:
seasons[0]

,Rk,Player,Tm,Age,G,GS,Cmp,Att,Cmp%,Yds,TD,Int,1D,Y/G,Sk
0,1,Drew Brees,NOR,32,16,16,468,657,71.2,5476,46,14,279,342.3,24
1,2,Tom Brady,NWE,34,16,16,401,611,65.6,5235,39,12,262,327.2,32
2,3,Matthew Stafford,DET,23,16,16,421,663,63.5,5038,41,16,243,314.9,36
3,4,Eli Manning,NYG,30,16,16,359,589,61.0,4933,29,16,218,308.3,28
4,5,Aaron Rodgers,GNB,28,15,15,343,502,68.3,4643,45,6,209,309.5,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,92,Adam Podlesh,CHI,28,16,0,0,1,0.0,0,0,0,0,0.0,0
92,93,Bernard Scott,CIN,27,16,1,0,1,0.0,0,0,0,0,0.0,0
93,94,Brad Smith,BUF,28,15,5,0,1,0.0,0,0,1,0,0.0,1
94,95,Micheal Spurlock,TAM,28,12,0,0,1,0.0,0,0,0,0,0.0,0
